# Big G Express EDA

## Import

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from my_functions import create_target_window, stratifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

## Read in

In [5]:
faults = pd.read_pickle('../data/faults_df.pickle')
diag = pd.read_pickle('../data/diag_df.pickle')
fdwide = pd.read_pickle('../data/fdwide_df.pickle')

## Use PCA to see if target values are meaningfully separated

In [8]:
full48 = create_target_window(fdwide, both_derate_types=False, target_window_hours=48)

In [22]:
full48['LampStatus'] = full48['LampStatus'].astype(int).astype(str)

array(['4351', '1279', '255', '17407', '21503', '1023', '5375', '18431',
       '2047', '5119', '22527', '6143', '65535', '2035', '544', '2', '0',
       '9', '11', '18419', '62463', '22515', '617', '63487', '16639',
       '17663', '511', '20735', '16895', '50175', '51199', '6131',
       '50431', '65523', '11801', '55295', '28436'], dtype=object)

In [23]:
full48['Throttle']

0           NaN
1           NaN
2           8.8
3           NaN
4          14.4
           ... 
1187330     NaN
1187331     0.0
1187332     NaN
1187333     0.0
1187334     NaN
Name: Throttle, Length: 1187335, dtype: float64

In [31]:
full48['timeTillLast'].dt.total_seconds()

0          38205420.0
1          38035435.0
2          36992502.0
3          36989631.0
4          36884516.0
              ...    
1187330      163614.0
1187331        2328.0
1187332           0.0
1187333        1797.0
1187334           0.0
Name: timeTillLast, Length: 1187335, dtype: float64

In [34]:
dropcols = ['ESS_Id','eventDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'faultValue', 'EquipmentID', 'MCTNumber', 'LocationTimeStamp', 'eventGroup', 'FaultId', 'EventTimeStamp']
catcols = ['spn', 'fmi', 'spn-fmi']
X = full48.assign(timeTillLast = full48['timeTillLast'].dt.total_seconds()).drop(columns = dropcols)

ct = ColumnTransformer([
            ('ohe', OneHotEncoder(sparse = False, drop = 'first'), catcols)
        ],
        remainder="passthrough"
    )

pipeline = Pipeline([
    ('ct', ct),
    ('scaler', StandardScaler()),
    ('pca', PCA())
])

pipeline.fit(X)

MemoryError: Unable to allocate 13.9 GiB for an array with shape (1187335, 1569) and data type object